In [8]:
"""
CryptoPanic API Wrapper.

Docs for API found here. https://cryptopanic.com/developers/api/

CryptoPanic Server is cached every 30 seconds. So crawling at a faster rate is pointless.
All API methods are rate limited per IP at 5req/sec.

Use like this:

    df = concatenate_pages(pages_list=get_pages_list_json(2, make_url(filter='hot', currencies='btc,eth,xrp')))

"""

import requests
import pandas as pd
import datetime
import config
import time

# main_script.py
import config

# Access the API key
api_key = config.API_KEY


global_api_rate_delay = .2  # All API methods are rate limited per IP at 5req/sec.


def make_url(filter=None, currencies=None, kind=None, region=None, page=None):
    """Handle of URL variables for API POST."""
    url = 'https://cryptopanic.com/api/v1/posts/?auth_token={}'.format(config.API_KEY)

    if currencies is not None:
        if len(currencies.split(',')) <= 50:
            url += "&currencies={}".format(currencies)
        else:
            print("Warning: Max Currencies is 50")
            return

    if kind is not None and kind in ['news', 'media']:
        url += "&kind={}".format(kind)

    filters = ['rising', 'hot', 'bullish', 'bearish', 'important', 'saved', 'lol']
    if filter is not None and filter in filters:
        url += "&filter={}".format(filter)

    regions = ['en', 'de', 'es', 'fr', 'it', 'pt', 'ru']  # (English), (Deutsch), (Español), (Français), (Italiano), (Português), (Русский)--> Respectively
    if region is not None and region in regions:
        url += "&region={}".format(region)

    if page is not None:
        url += "&page={}".format(page)

    return url


def get_page_json(url=None):
    """
    Get First Page.

    Returns Json.

    """
    time.sleep(global_api_rate_delay)
    if not url:
        url = "https://cryptopanic.com/api/v1/posts/?auth_token={}".format(config.API_KEY)
    page = requests.get(url)
    data = page.json()
    return data


def get_pages_list_json(lookback, url):
    """
    Get history of pages starting from page 1 to the lookback.

    Returns: List of Pages in Json format

    """
    pages_list_json = [get_page_json(url)]

    for i in range(lookback):
        pages_list_json.append(get_page_json(pages_list_json[i]["next"]))

    return pages_list_json


def get_df(data):
    """Return pandas DF."""
    df = pd.DataFrame(data)
    try:
        df['created_at'] = pd.to_datetime(df.created_at)
    except Exception as e:
        pass

    return df


def concatenate_pages(pages_list):
    """Concatenate Pages into one Dataframe."""
    frames = []
    for page in pages_list:
        frames.append(get_df(page))

    return pd.concat(frames, ignore_index=True)

# df = concatenate_pages(get_pages_list_json(2, make_url()))

df = concatenate_pages(pages_list=get_pages_list_json(2, make_url(filter='hot', currencies='btc')))
df

,count,next,previous,results
0,200,https://cryptopanic.com/api/v1/posts/?auth_tok...,None,"{'kind': 'news', 'domain': 'en.bitcoinsistemi...."
1,200,https://cryptopanic.com/api/v1/posts/?auth_tok...,None,"{'kind': 'news', 'domain': 'newsbtc.com', 'vot..."
2,200,https://cryptopanic.com/api/v1/posts/?auth_tok...,None,"{'kind': 'news', 'domain': 'newsbtc.com', 'vot..."
3,200,https://cryptopanic.com/api/v1/posts/?auth_tok...,None,"{'kind': 'news', 'domain': 'cryptobriefing.com..."
4,200,https://cryptopanic.com/api/v1/posts/?auth_tok...,None,"{'kind': 'news', 'domain': 'en.bitcoinsistemi...."
5,200,https://cryptopanic.com/api/v1/posts/?auth_tok...,None,"{'kind': 'news', 'domain': 'decrypt.co', 'vote..."
6,200,https://cryptopanic.com/api/v1/posts/?auth_tok...,None,"{'kind': 'news', 'domain': 'finbold.com', 'vot..."
7,200,https://cryptopanic.com/api/v1/posts/?auth_tok...,None,"{'kind': 'news', 'domain': 'coinpaprika.com', ..."
8,200,https://cryptopanic.com/api/v1/posts/?auth_tok...,None,"{'kind': 'news', 'domain': 'en.bitcoinsistemi...."
9,200,https://cryptopanic.com/api/v1/posts/?auth_tok...,None,"{'kind': 'news', 'domain': 'feeds2.benzinga.co..."


In [12]:
import requests
import pandas as pd
import datetime
import config
import time

# main_script.py
import config

# Access the API key
api_key = config.API_KEY

def make_url(filter=None, currencies=None, kind=None, region=None, page=None):
    """Handle of URL variables for API POST."""
    url = 'https://cryptopanic.com/api/v1/posts/?auth_token={}'.format(config.API_KEY)

    if currencies is not None:
        if len(currencies.split(',')) <= 50:
            url += "&currencies={}".format(currencies)
        else:
            print("Warning: Max Currencies is 50")
            return

    if kind is not None and kind in ['news', 'media']:
        url += "&kind={}".format(kind)

    filters = ['rising', 'hot', 'bullish', 'bearish', 'important', 'saved', 'lol']
    if filter is not None and filter in filters:
        url += "&filter={}".format(filter)

    regions = ['en', 'de', 'es', 'fr', 'it', 'pt', 'ru']  # (English), (Deutsch), (Español), (Français), (Italiano), (Português), (Русский)--> Respectively
    if region is not None and region in regions:
        url += "&region={}".format(region)

    if page is not None:
        url += "&page={}".format(page)

    return url


def get_page_json(url=None):
    """
    Get First Page.

    Returns Json.

    """
    time.sleep(global_api_rate_delay)
    if not url:
        url = "https://cryptopanic.com/api/v1/posts/?auth_token={}".format(config.API_KEY)
    page = requests.get(url)
    data = page.json()
    return data


def get_pages_list_json(lookback, url):
    """
    Get history of pages starting from page 1 to the lookback.

    Returns: List of Pages in Json format

    """
    pages_list_json = [get_page_json(url)]

    for i in range(lookback):
        pages_list_json.append(get_page_json(pages_list_json[i]["next"]))

    return pages_list_json

def get_df(data):
    """Return pandas DF."""
    # Ensure that data is a list of dictionaries
    if not all(isinstance(item, dict) for item in data):
        raise ValueError("Data must be a list of dictionaries")
    df = pd.DataFrame(data)
    try:
        df['created_at'] = pd.to_datetime(df['created_at'])
    except Exception as e:
        print(f"An error occurred: {e}")
    return df

def concatenate_pages(pages_list):
    """Concatenate Pages into one Dataframe."""
    # Ensure that pages_list is a list of lists of dictionaries
    if not all(isinstance(page, list) and all(isinstance(item, dict) for item in page) for page in pages_list):
        raise ValueError("Pages list must be a list of lists of dictionaries")
    frames = [get_df(page) for page in pages_list]
    return pd.concat(frames, ignore_index=True)

# The get_last_posts function needs to be updated to pass a list of lists of dictionaries
def get_last_posts(api_key, number_of_posts=200):
    pages_list = []
    page = 1
    total_results = 0
    while total_results < number_of_posts:
        url = make_url(api_key=api_key, filter='important', currencies='BTC', page=page)
        data = get_page_json(url)
        page_results = data['results']
        pages_list.append(page_results)
        total_results += len(page_results)
        page += 1
        if 'next' not in data or not data['next']:
            break  # No more pages to fetch
    # Trim the list to contain exactly the number_of_posts results
    flat_list = [item for sublist in pages_list for item in sublist]
    flat_list = flat_list[:number_of_posts]
    return [flat_list]

# Get the last 200 posts
pages_list = get_last_posts(api_key, number_of_posts=200)
df_last_200_posts = concatenate_pages(pages_list)
print(df_last_200_posts.head())


   kind                 domain  \
0  news          dailyhodl.com   
1  news     cryptobriefing.com   
2  news                u.today   
3  news            newsbtc.com   
4  news  en.bitcoinsistemi.com   

                                               votes  \
0  {'negative': 0, 'positive': 7, 'important': 4,...   
1  {'negative': 0, 'positive': 10, 'important': 5...   
2  {'negative': 0, 'positive': 5, 'important': 3,...   
3  {'negative': 6, 'positive': 10, 'important': 3...   
4  {'negative': 1, 'positive': 35, 'important': 1...   

                                              source  \
0  {'title': 'The Daily Hodl', 'region': 'en', 'd...   
1  {'title': 'CryptoBriefing', 'region': 'en', 'd...   
2  {'title': 'U.Today', 'region': 'en', 'domain':...   
3  {'title': 'NewsBTC', 'region': 'en', 'domain':...   
4  {'title': 'en bitcoinsistemi', 'region': 'en',...   

                                               title          published_at  \
0  $100,000,000,000 in TradFi Money Waiting

In [13]:
df_last_200_posts

,kind,domain,votes,source,title,published_at,slug,currencies,id,url,created_at
0,news,dailyhodl.com,"{'negative': 0, 'positive': 7, 'important': 4,...","{'title': 'The Daily Hodl', 'region': 'en', 'd...","$100,000,000,000 in TradFi Money Waiting for B...",2023-11-20T19:05:54Z,100000000000-in-TradFi-Money-Waiting-for-Bitco...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19051010,https://cryptopanic.com/news/19051010/10000000...,2023-11-20 19:05:54+00:00
1,news,cryptobriefing.com,"{'negative': 0, 'positive': 10, 'important': 5...","{'title': 'CryptoBriefing', 'region': 'en', 'd...","ARK Ignores SEC Advice, Refiles In-Kind Bitcoi...",2023-11-20T15:39:10Z,ARK-Ignores-SEC-Advice-Refiles-In-Kind-Bitcoin...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19050524,https://cryptopanic.com/news/19050524/ARK-Igno...,2023-11-20 15:39:10+00:00
2,news,u.today,"{'negative': 0, 'positive': 5, 'important': 3,...","{'title': 'U.Today', 'region': 'en', 'domain':...","XRP, BTC, ETH, ADA, SOL Funds Attract More Inf...",2023-11-20T15:39:00Z,XRP-BTC-ETH-ADA-SOL-Funds-Attract-More-Inflows...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19050530,https://cryptopanic.com/news/19050530/XRP-BTC-...,2023-11-20 15:39:00+00:00
3,news,newsbtc.com,"{'negative': 6, 'positive': 10, 'important': 3...","{'title': 'NewsBTC', 'region': 'en', 'domain':...",ADA Price (Cardano) Breaking This Confluence R...,2023-11-20T04:28:28Z,ADA-Price-Cardano-Breaking-This-Confluence-Res...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19049104,https://cryptopanic.com/news/19049104/ADA-Pric...,2023-11-20 04:28:28+00:00
4,news,en.bitcoinsistemi.com,"{'negative': 1, 'positive': 35, 'important': 1...","{'title': 'en bitcoinsistemi', 'region': 'en',...",JUST IN: Pro-Bitcoin Candidate Javier Milei Wi...,2023-11-19T23:40:06Z,JUST-IN-Pro-Bitcoin-Candidate-Javier-Milei-Win...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19048931,https://cryptopanic.com/news/19048931/JUST-IN-...,2023-11-19 23:40:06+00:00
...,...,...,...,...,...,...,...,...,...,...,...
195,news,finbold.com,"{'negative': 2, 'positive': 1, 'important': 3,...","{'title': 'Finbold', 'region': 'en', 'domain':...",Why is the crypto market down today?,2023-09-25T12:01:37Z,Why-is-the-crypto-market-down-today,"[{'code': 'MC_', 'title': 'Market Cap', 'slug'...",18926310,https://cryptopanic.com/news/18926310/Why-is-t...,2023-09-25 12:01:37+00:00
196,news,finbold.com,"{'negative': 2, 'positive': 4, 'important': 7,...","{'title': 'Finbold', 'region': 'en', 'domain':...",Stock market to crash? Bankruptcy filings soar...,2023-09-25T10:48:31Z,Stock-market-to-crash-Bankruptcy-filings-soar-...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",18926101,https://cryptopanic.com/news/18926101/Stock-ma...,2023-09-25 10:48:31+00:00
197,news,dailyhodl.com,"{'negative': 1, 'positive': 3, 'important': 3,...","{'title': 'The Daily Hodl', 'region': 'en', 'd...",Chainlink (LINK) and Three Low-Cap Altcoins Co...,2023-09-25T00:00:45Z,Chainlink-LINK-and-Three-Low-Cap-Altcoins-Coul...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",18925240,https://cryptopanic.com/news/18925240/Chainlin...,2023-09-25 00:00:45+00:00
198,news,en.bitcoinsistemi.com,"{'negative': 0, 'positive': 5, 'important': 4,...","{'title': 'en bitcoinsistemi', 'region': 'en',...",Coinbase’s Bitcoin Reserve Wallet Discovered: ...,2023-09-22T18:19:07Z,Coinbases-Bitcoin-Reserve-Wallet-Discovered-He...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",18922568,https://cryptopanic.com/news/18922568/Coinbase...,2023-09-22 18:19:07+00:00


In [14]:
# Write the DataFrame to a CSV file
df_last_200_posts.to_csv('crypto_panic_2.csv', index=False)